# 0. Setup

In [1]:
import pandas as pd
import requests
import json
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import os
import yfinance as yf

# 1. API Request

In [2]:
url = 'https://api.hkma.gov.hk/public/market-data-and-statistics/daily-monetary-statistics/daily-figures-interbank-liquidity?offset=0&pagesize=999&sortby=end_of_date&sortorder=desc'
response = requests.get(url).text
response = json.loads(response)
df = pd.DataFrame(response['result']['records'])
df['end_of_date'] = pd.to_datetime(df['end_of_date'])

In [3]:
df

,end_of_date,cu_weakside,cu_strongside,disc_win_base_rate,hibor_overnight,hibor_fixing_1m,twi,opening_balance,closing_balance,market_activities,...,forex_trans_t4,other_market_activities_t4,reversal_of_discount_window_t4,interest_payment_issuance_efbn_t4,forecast_aggregate_bal_t4,forex_trans_u,other_market_activities_u,reversal_of_discount_window_u,interest_payment_issuance_efbn_u,forecast_aggregate_bal_u
0,2025-08-20,7.85,7.75,4.75,2.63,2.85774,103.8,53716,53909,+0,...,None,None,None,None,NaN,+0,+0,-0,-620,53311
1,2025-08-19,7.85,7.75,4.75,2.63,2.57417,103.7,53716,53716,+0,...,None,None,None,None,NaN,+0,+0,-0,-566,53311
2,2025-08-18,7.85,7.75,4.75,1.88,2.01393,103.4,53716,53716,+0,...,None,None,None,None,NaN,+0,+0,-0,-355,53311
3,2025-08-15,7.85,7.75,4.75,0.83,1.45095,103.4,57091,53716,-3376,...,None,None,None,None,NaN,+0,+0,-0,-335,53311
4,2025-08-14,7.85,7.75,4.75,0.32,1.04524,102.9,64156,57091,-7065,...,None,None,None,None,NaN,+0,+0,-0,-404,53311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2021-08-10,7.85,7.75,0.50,0.01,0.07429,101.6,457457,457457,0,...,None,None,None,None,NaN,0,0,0,-2,457454
995,2021-08-09,7.85,7.75,0.50,0.01,0.07661,101.5,457457,457457,0,...,None,None,None,None,NaN,0,0,0,-1,457454
996,2021-08-06,7.85,7.75,0.50,0.01,0.07768,101.4,457457,457457,0,...,None,None,None,None,NaN,0,0,0,-1,457454
997,2021-08-05,7.85,7.75,0.50,0.01,0.07768,101.3,457457,457457,0,...,None,None,None,None,NaN,0,0,0,-3,457454


In [9]:
# HKAB API URLs for yield data
def get_hibor(startDate, endDate)->pd.DataFrame:
    selected_keys = ['Overnight', '1 Week', '2 Weeks', '1 Month', '2 Months', '3 Months', '6 Months', '12 Months']
    hibor_df = pd.DataFrame(columns=selected_keys)
    for date in pd.date_range(startDate, endDate):
        # print(date)
        if date.dayofweek <5:
            year = date.year
            month = date.month
            day = date.day
            hibor_url = f"https://www.hkab.org.hk/api/hibor?year={year}&month={month}&day={day}"
            hibor_response = requests.get(hibor_url).json()
            if hibor_response['isHoliday']==False:
                temp_df = pd.DataFrame(hibor_response, index=[date], columns=selected_keys)
                hibor_df = pd.concat([hibor_df, temp_df], ignore_index=False)
    hibor_df=hibor_df.dropna().sort_index(ascending=False).reset_index(drop=False).rename(columns={'index':'date'})
    return hibor_df

In [10]:
endDate = pd.Timestamp.now(tz='Asia/Hong_Kong').strftime('%Y-%m-%d')
startDate =(pd.Timestamp.now(tz='Asia/Hong_Kong') - pd.Timedelta(days=365)).strftime('%Y-%m-%d')
hibor_df = get_hibor(startDate, endDate)
hibor_df

/var/folders/h_/dmgktz0n27g262j_0g_v4qvm0000gn/T/ipykernel_30957/2616980000.py:15: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  hibor_df = pd.concat([hibor_df, temp_df], ignore_index=False)
/var/folders/h_/dmgktz0n27g262j_0g_v4qvm0000gn/T/ipykernel_30957/2616980000.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hibor_df = pd.concat([hibor_df, temp_df], ignore_index=False)
/var/folders/h_/dmgktz0n27g262j_0g_v4qvm0000gn/T/ipykernel_30957/2616980000.py:15: FutureWarning: The behavior of DataFrame concatenation with empty

,date,Overnight,1 Week,2 Weeks,1 Month,2 Months,3 Months,6 Months,12 Months
0,2025-08-21,2.78762,2.80000,2.87917,2.85774,2.85554,2.92595,3.00000,3.18405
1,2025-08-20,2.78762,2.80000,2.87917,2.85774,2.85554,2.92595,3.00000,3.18405
2,2025-08-19,2.89393,2.66119,2.67393,2.57417,2.57774,2.57488,2.69845,2.93393
3,2025-08-18,1.96768,1.84381,1.92143,2.01393,2.09583,2.18333,2.46345,2.76637
4,2025-08-15,0.76935,0.94941,1.12696,1.45095,1.64393,1.86060,2.38411,2.67143
...,...,...,...,...,...,...,...,...,...
240,2024-08-27,3.09917,3.82321,3.82917,3.92298,4.14149,4.22179,4.28577,4.33060
241,2024-08-26,3.21464,3.86476,3.89429,3.97345,4.17131,4.25071,4.30000,4.35524
242,2024-08-23,3.34571,3.70000,3.91780,3.98530,4.21280,4.27012,4.30000,4.34643
243,2024-08-22,3.41000,3.67631,3.91637,3.99714,4.21494,4.28315,4.30000,4.36095


# 2.0 HIBOR & Liquidity

## 2.1 HIBOR (Overnight & 1-Month)

In [ ]:
hibor_df = hibor_df.head(251)

In [21]:
hibor_fig = px.line(hibor_df,
             x='date', y=['Overnight', '1 Week', '1 Month', '3 Months', '6 Months', '12 Months'],
             title='HIBOR Rates',
             labels={'end_of_date': 'Date', 'value': 'HIBOR Rate (%)'},
             )

last_date = hibor_df['date'].iloc[0].strftime('%Y-%m-%d')
last_overnight = round(hibor_df['Overnight'].iloc[0],2)
last_1m = round(hibor_df['1 Month'].iloc[0], 2)
last_3m = round(hibor_df['3 Months'].iloc[0], 2)
last_6m = round(hibor_df['6 Months'].iloc[0], 2)
last_12m = round(hibor_df['12 Months'].iloc[0], 2)

hibor_fig.add_annotation(
    text=f"Date: {last_date}<br>Overnight: {last_overnight}%<br>1M: {last_1m}%<br>3M: {last_3m}%<br>6M: {last_6m}%<br>12M: {last_12m}%",
    xref="paper", yref="paper",
    x=1, y=1.2,  
    showarrow=False,
    bgcolor="rgba(1, 108, 2, 1)",
    borderwidth=2,
    font=dict(size=12, color="white"),
    align='right',
)

hibor_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

hibor_fig.show()

## 2.2 Aggregate Balance Open & Close

In [ ]:
aggreBal_df = df[['end_of_date', 'opening_balance', 'closing_balance']].head(251)
aggreBal_df['day_change'] = aggreBal_df['closing_balance'] - aggreBal_df['opening_balance']
aggreBal_df['high'] = aggreBal_df[['opening_balance', 'closing_balance']].max(axis=1)
aggreBal_df['low'] = aggreBal_df[['opening_balance', 'closing_balance']].min(axis=1)
aggreBal_df

In [ ]:
aggreBal_fig = go.Figure(data=[go.Candlestick(
    x=aggreBal_df['end_of_date'],
    open=aggreBal_df['opening_balance'],
    high=aggreBal_df['high'],
    low=aggreBal_df['low'],
    close=aggreBal_df['closing_balance'],
    name='Aggregate Balance',
    showlegend=True,
)])

last_date = aggreBal_df['end_of_date'].iloc[0]
last_open = aggreBal_df['opening_balance'].iloc[0]
last_close = aggreBal_df['closing_balance'].iloc[0]
last_day_change = aggreBal_df['day_change'].iloc[0]

aggreBal_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>Open: {last_open}<br>Close: {last_close}<br>Change: {last_day_change}",
    xref="paper", yref="paper",
    x=1, y=1.2,
    showarrow=False,
    bgcolor="rgba(0, 78, 123, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

aggreBal_fig.update_layout(
    xaxis_rangeslider_visible=True,
    title="Hong Kong Aggregate Balance (Candlestick)",
    xaxis_title="Date",
    yaxis_title="Balance (HKD Million)",
)
aggreBal_fig.show()

# 3.0 FX

# 3.1 Convertibility Undertaking (CU)

In [ ]:
usdhkd = yf.Ticker("HKD=X")
usdhkd_df = usdhkd.history(period ='max').reset_index()
usdhkd_df['Date'] = pd.to_datetime(usdhkd_df['Date'], format='%Y-%m-%d')
usdhkd_df = usdhkd_df[['Date', 'Close']]
usdhkd_df = usdhkd_df.rename(columns={'Date': 'end_of_date', 'Close': 'usdhkd_close'})
usdhkd_df['end_of_date'] = pd.to_datetime(usdhkd_df['end_of_date'])
usdhkd_df.set_index('end_of_date', inplace=True)
usdhkd_df.index = usdhkd_df.index.tz_localize(None)

In [ ]:
cu_df = df[['end_of_date', 'cu_weakside', "cu_strongside"]].head(251)
cu_df['end_of_date'] = pd.to_datetime(cu_df['end_of_date'])
cu_df = cu_df.set_index('end_of_date').join(usdhkd_df, on='end_of_date', how='left')

In [ ]:
cu_df = cu_df.reset_index()
cu_df.sort_values(by='end_of_date', ascending=False, inplace=True)
cu_df

In [ ]:
cu_fig = px.line(cu_df,
             x='end_of_date', y=['usdhkd_close', 'cu_weakside', 'cu_strongside'],
             title='USD/HKD and Currency Pegs',
             labels={'end_of_date': 'Date', 'value': 'Value (HKD)'},
             )


last_date = cu_df['end_of_date'].iloc[0]
last_usdhkd_close = round(cu_df['usdhkd_close'].iloc[0],4)

cu_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>usdhkd: {last_usdhkd_close}",
    xref="paper", yref="paper",
    x=1, y=1.2, 
    showarrow=False,
    bgcolor="rgba(108, 1, 2, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

cu_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

cu_fig.show()

# 3.2 Trade-Weighted Index, TWI

In [ ]:
hkdtwi_df = df[['end_of_date', 'twi']].head(251)

In [ ]:
hkdtwi_fig = px.line(hkdtwi_df,
             x='end_of_date', y='twi',
             title='HKD Trade-Weighted Index (TWI)',
             labels={'end_of_date': 'Date', 'value': 'Value'},
             )


last_date = hkdtwi_df['end_of_date'].iloc[0]
last_twi = hkdtwi_df['twi'].iloc[0]

hkdtwi_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>TWI(HKD): {last_twi}",
    xref="paper", yref="paper",
    x=1, y=1.2,
    showarrow=False,
    bgcolor="rgba(156, 33, 315, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

hkdtwi_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

hkdtwi_fig.show()

# 4.0 Stock Market

In [ ]:
hsi = yf.Ticker("^HSI")
hsi_df = hsi.history(period ='251d').reset_index()
hsi_df.set_index('Date', inplace=True)
hsi_df.index = hsi_df.index.tz_localize(None)
hsi_df.sort_values(by='Date', ascending=False, inplace=True)
hsi_df['day_change'] = hsi_df['Close'] - hsi_df['Open']
hsi_df = hsi_df.reset_index()
hsi_df

In [ ]:
hsi_fig = px.line(hsi_df,
             x='Date', y='Close',
             title='HSI',
             labels={'Date': 'Date', 'value': 'Value'},
             )


last_date = hsi_df['Date'].iloc[0]
last_hsi = round(hsi_df['Close'].iloc[0],2)

hsi_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>Index(HKD): {last_hsi}",
    xref="paper", yref="paper",
    x=1, y=1.2, 
    showarrow=False,
    bgcolor="rgba(156, 33, 315, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

hsi_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

hsi_fig.show()

In [ ]:
# hibor_fig.write_html('assets/hibor_fig.html')
# aggreBal_fig.write_html('assets/aggreBal_fig.html')
# cu_fig.write_html('assets/cu_fig.html')
# hkdtwi_fig.write_html('assets/hkdtwi_fig.html')
# hsi_fig.write_html('assets/hsi_fig.html')

In [ ]:
# figs = [
#     (hsi_fig, "HSI"),
#     (hibor_fig, "HIBOR"),
#     (aggreBal_fig, "AggreBal"),
#     (cu_fig, "Currency"),
#     (hkdtwi_fig, "HKDTWI"),
# ]

# html = """<!DOCTYPE html>
# <html>
# <head>
#     <meta charset="UTF-8">
#     <title>Charts</title>
#     <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
#     <style>
#         body { margin: 20px; text-align: center; }
#         div { margin: 30px auto; width: 80%; }
#     </style>
# </head>
# <body>
#     <h1>Charts</h1>
# """
# for i, (fig, title) in enumerate(figs, 1):
#     html += "<hr>\n"
#     html += f"<h2>{title}</h2>\n"
#     html += fig.to_html(full_html=False, div_id=f"chart{i}", include_plotlyjs=True)

# html += "</body>\n</html>"
# output_path = os.path.abspath(os.path.join(os.path.abspath(os.path.join(os.getcwd(), "..")), ".."))+ "/docs/charts.html"
# with open(output_path, "w", encoding="utf-8") as f:
#     f.write(html)